In [28]:
import pandas as pd
path_to_data_dir = "../../data/"

targets = pd.read_csv(path_to_data_dir + "target.csv").set_index(["date", "ticker"])
sp_500 = pd.read_csv(path_to_data_dir + "sp_500.csv", sep=", ").set_index("Date")

/var/folders/tq/51dxx6813y7g8dg44h973dmr0000gn/T/ipykernel_62352/1857336199.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sp_500 = pd.read_csv(path_to_data_dir + "sp_500.csv", sep=", ").set_index("Date")


In [30]:
sp_500.index = pd.to_datetime(sp_500.index, format = "%m/%d/%y").strftime("%Y-%m-%d")

In [31]:
sp_500 = sp_500.sort_index()
sp_500.head()

,Open,High,Low,Close
Date,,,,
2023-09-01,4530.85,4541.25,4501.35,4515.77
2023-09-05,4510.06,4514.29,4496.01,4496.83
2023-09-06,4490.35,4490.35,4442.38,4465.48
2023-09-07,4434.55,4457.81,4430.46,4451.14
2023-09-08,4451.30,4473.53,4448.38,4457.49


In [32]:
import pandas_market_calendars as mcal
import datetime
def getPreviousMarketOpenDays(date: str, n: int):
	# Create a calendar
	nyse = mcal.get_calendar('NYSE')
	day = datetime.datetime.strptime(date, "%Y-%m-%d")
	schedule = nyse.schedule(
		start_date = (day - datetime.timedelta(days = 20 + n*2)).strftime("%Y-%m-%d"),
		end_date=day.strftime("%Y-%m-%d"))
	
	return schedule.index.strftime("%Y-%m-%d").tolist()[-n:]

In [33]:
all_datetimes = targets.index.get_level_values(0).unique()

pct_change_col = []

for dt in sp_500.index.sort_values()[5:]:
	prev_datetime = getPreviousMarketOpenDays(dt, 6)[0]

	pct_change = (sp_500.loc[dt, "Close"] / sp_500.loc[prev_datetime, "Close"]) - 1.
	pct_change_col.append(pct_change)

pct_change_col[:5]

[-0.006269141253872679,
 -0.007767694131199154,
 0.0004389225794314555,
 0.012122737096563974,
 -0.001608528566525158]

In [34]:
sp_500 = sp_500.iloc[:-5]
sp_500["5_day_pct_change"] = pct_change_col

In [35]:
sp_500.head()

,Open,High,Low,Close,5_day_pct_change
Date,,,,,
2023-09-01,4530.85,4541.25,4501.35,4515.77,-0.006269
2023-09-05,4510.06,4514.29,4496.01,4496.83,-0.007768
2023-09-06,4490.35,4490.35,4442.38,4465.48,0.000439
2023-09-07,4434.55,4457.81,4430.46,4451.14,0.012123
2023-09-08,4451.30,4473.53,4448.38,4457.49,-0.001609


In [36]:
sp_change_col = []
for dt in targets.index.get_level_values(0):
	sp_change_col.append(sp_500.loc[dt, "5_day_pct_change"])
sp_change_col[:2]

[-0.009654161918273396, -0.009654161918273396]

In [37]:
targets["s&p_change"] = sp_change_col
targets.head()

target  s&p_change
date       ticker                      
2023-09-28 AEO     0.079344   -0.009654
           AAPL    0.009886   -0.009654
           AEE    -0.052310   -0.009654
           AEP    -0.039953   -0.009654
           AES    -0.167718   -0.009654

In [38]:
targets["change_adj_s&p"] = targets["target"] - targets["s&p_change"]
targets.head()

target  s&p_change  change_adj_s&p
date       ticker                                      
2023-09-28 AEO     0.079344   -0.009654        0.088998
           AAPL    0.009886   -0.009654        0.019540
           AEE    -0.052310   -0.009654       -0.042656
           AEP    -0.039953   -0.009654       -0.030299
           AES    -0.167718   -0.009654       -0.158063

In [40]:
targets.to_csv(path_to_data_dir + "s&p_adjusted_target.csv", index = True)